In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from graspy.embed import AdjacencySpectralEmbed
from graspy.plot import pairplot
from graspy.embed import OutOfSampleAdjacencySpectralEmbed as OOSASE
import graspy

import networkx as nx

import _pickle as pkl
# Need OOS

In [3]:
np.random.seed(1)
reload_data = False

if reload_data:
    nodes = pd.read_csv('nodes', delimiter='\t')
    print(nodes.head())

    edges = pd.read_csv('edges', delimiter='\t',header=None)
    print(edges.head())

    G = nx.from_pandas_edgelist(edges, source=0, target=1)
    lcc = max(nx.connected_components(G), key=len)
    G_lcc = G.subgraph(lcc).copy()
    pkl.dump(G_lcc, open('nas_graph.pkl', 'wb'))
    
    N = len(G_lcc)
    in_sample_prop = 0.2

    nodes = pd.read_csv('nodes', delimiter='\t')['hash']

    in_sample_id = np.random.choice(nodes, int(N*0.2))

    temp_G = G_lcc.subgraph(in_sample_id).copy()
    lcc = max(nx.connected_components(temp_G))

    in_sample_G = temp_G.subgraph(lcc).copy()

    pkl.dump(in_sample_G, open('in_sample_G.pkl', 'wb'))
else:
    G_lcc = pkl.load(open('nas_graph.pkl', 'rb'))
    in_sample_G = pkl.load(open('in_sample_G.pkl', 'rb'))
    
print('largest connected component:' + str(len(G_lcc)))
print('in sample size:' + str(len(in_sample_G)))

                               hash  mean_final_validation_accuracy  \
0  00005c142e6f48ac74fdcf73e3439874                        0.926416   
1  0000718c96730a92ea6d5242f8ce77e2                        0.894631   
2  0000cb5372fad9d62c470df699ac6d52                        0.927350   
3  0000fa05697179112aaf69c2f0a51a0f                        0.909756   
4  00010e19334eb9dcbe1fe61a98d524c2                        0.897169   

   mean_final_training_time  
0               1768.784953  
1               1206.150024  
2               1343.049967  
3               1302.710978  
4               1251.861003  
                                  0                                 1
0  00005c142e6f48ac74fdcf73e3439874  09fbbe5d0e151746fee03a1e35c652eb
1  00005c142e6f48ac74fdcf73e3439874  3c13829b7c2e926ae84e149eb51ed37a
2  00005c142e6f48ac74fdcf73e3439874  636c82bf88337fda92e8a6d312b8ae91
3  00005c142e6f48ac74fdcf73e3439874  65da9c92894ce3c57c1d9edde81a72a6
4  00005c142e6f48ac74fdcf73e3439874  6d0dc3

In [3]:




N = len(G_lcc)
in_sample_prop = 0.2

nodes = pd.read_csv('nodes', delimiter='\t')['hash']

in_sample_id = np.random.choice(nodes, int(N*0.2))

temp_G = G_lcc.subgraph(in_sample_id).copy()
lcc = max(nx.connected_components(temp_G))

in_sample_G = temp_G.subgraph(lcc).copy()

pkl.dump(in_sample_G, open('in_sample_G', 'wb'))

In [4]:
len(in_sample_G)

46759

In [5]:
oos_ase = OOSASE(in_sample_proportion=1, connected_attempts=100, check_lcc=False)
oos_ase.fit(in_sample_G)

begin svd


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/mnt/c/Users/hhelm/virtualenvs/hh/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-c7cec997fbb6>", line 2, in <module>
    oos_ase.fit(in_sample_G)
  File "/mnt/c/Users/hhelm/Desktop/NeuroData/graspy/graspy/embed/oosase.py", line 208, in fit
    self._reduce_dim(in_sample_A)
  File "/mnt/c/Users/hhelm/Desktop/NeuroData/graspy/graspy/embed/base.py", line 90, in _reduce_dim
    n_iter=self.n_iter,
  File "/mnt/c/Users/hhelm/Desktop/NeuroData/graspy/graspy/embed/svd.py", line 235, in selectSVD
    elbows, _ = select_dimension(X, n_elbows=n_elbows, threshold=None)
  File "/mnt/c/Users/hhelm/Desktop/NeuroData/graspy/graspy/embed/svd.py", line 143, in select_dimension
    D = scipy.sparse.linalg.svds(A=X, k=k, return_singular_vectors=False)
  File "/mnt/c/Users/hhelm/virtualenvs/hh/lib/python3.6/site-packages/scipy/sparse/linalg/eigen/

KeyboardInterrupt: 

In [ ]:
in_sample_id = np.random.choice(nodes, int(N*0.2))

In [ ]:
from graspy.utils import is_fully_connected
is_fully_connected(G_lcc)

In [13]:
list(G_lcc.edges(['09fbbe5d0e151746fee03a1e35c652eb']))

[('09fbbe5d0e151746fee03a1e35c652eb', '00005c142e6f48ac74fdcf73e3439874'),
 ('09fbbe5d0e151746fee03a1e35c652eb', '316b39c0cfe5c93b7aa9e7bf28964372'),
 ('09fbbe5d0e151746fee03a1e35c652eb', '3c13829b7c2e926ae84e149eb51ed37a'),
 ('09fbbe5d0e151746fee03a1e35c652eb', '88ce8989ee0b3e71eb2f907fc5134465'),
 ('09fbbe5d0e151746fee03a1e35c652eb', 'a695ff736aa0cf94f92cacb315bf1fbd'),
 ('09fbbe5d0e151746fee03a1e35c652eb', 'c0b8db92277097333fb14c2c1e9583fd'),
 ('09fbbe5d0e151746fee03a1e35c652eb', 'c5a1ed0921a7bdeb92096368b08ce0a9'),
 ('09fbbe5d0e151746fee03a1e35c652eb', 'c768d47a3c80186120c503992b43fc9a'),
 ('09fbbe5d0e151746fee03a1e35c652eb', 'ca7cf8474d1691fdff731acab7b5be2f'),
 ('09fbbe5d0e151746fee03a1e35c652eb', 'cd55cd77ca7f85b5b6ef6dec4671781e'),
 ('09fbbe5d0e151746fee03a1e35c652eb', 'd3f5a49e76d22313d88b37105ed9408c'),
 ('09fbbe5d0e151746fee03a1e35c652eb', 'dcc19295b09da40d8e9a88b3a55d0875')]

In [17]:
G_lcc.get_edge_data('09fbbe5d0e151746fee03a1e35c652eb', '00005c142e6f48ac74fdcf73e343984')